* import library

In [3]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from pytz import timezone
from pandas import json_normalize
from bs4 import BeautifulSoup
#%run Wikipedia.ipynb


# Airport

In [2]:
def get_airport_info(latitude, longitude):
    
    url = "https://aerodatabox.p.rapidapi.com/airports/search/location"

    querystring = {
        "lat": str(latitude),
        "lon": str(longitude),
        "radiusKm": "50",
        "limit": "10",
        "withFlightInfoOnly": "True"
    }

    headers = {
        "X-RapidAPI-Key": "14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4",
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        return response.json()
    else:
        return None
    

## airport_info function

In [7]:
def get_airport_info(latitude, longitude):
    url = "https://aerodatabox.p.rapidapi.com/airports/search/location"

    querystring = {
        "lat": str(latitude),
        "lon": str(longitude),
        "radiusKm": "50",
        "limit": "10",
        "withFlightInfoOnly": "false"
    }

    headers = {
        "X-RapidAPI-Key": "14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4",
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        return response.json()
    else:
        return None

# Example usage
latitude = 48.1375
longitude = 11.575
airport_info = get_airport_info(latitude, longitude)
if airport_info:
    print(airport_info)
else:
    print("Failed to fetch airport information")

{'searchBy': {'lat': 48.1375, 'lon': 11.575}, 'count': 3, 'items': [{'icao': 'EDMO', 'iata': 'OBF', 'name': '', 'location': {'lat': 48.0814, 'lon': 11.2831}, 'countryCode': 'DE'}, {'icao': 'ETSF', 'iata': 'FEL', 'name': 'Fürstenfeldbruck ', 'shortName': 'Fürstenfeldbruck', 'municipalityName': 'Fürstenfeldbruck', 'location': {'lat': 48.205555, 'lon': 11.266943}, 'countryCode': 'DE'}, {'icao': 'EDDM', 'iata': 'MUC', 'name': 'Munich ', 'shortName': 'Munich', 'municipalityName': 'Munich', 'location': {'lat': 48.3538, 'lon': 11.7861}, 'countryCode': 'DE'}]}


In [11]:
# import Wikipedia as Wi

def get_airport_data(cities):
    airport_data = []

    for city in cities:
        # Retrieve coordinates for the city
        latitude, longitude = extract_coordinates(city)
        
        # Get airport information using the coordinates
        url = "https://aerodatabox.p.rapidapi.com/airports/search/location"
        querystring = {
            "lat": str(latitude),
            "lon": str(longitude),
            "radiusKm": "50",
            "limit": "10",
            "withFlightInfoOnly": "True"
        }
        headers = {
            "X-RapidAPI-Key": "14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4",
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }
        response = requests.get(url, headers=headers, params=querystring)
        
        # Append city and airport information to the list
        if response.status_code == 200:
            airport_info = response.json()
            if isinstance(airport_info['items'], list):
                airport_data.extend([{
                    'City': city,
                    'Airport_Info': item
                } for item in airport_info['items']])
        else:
            print(f"Failed to retrieve airport information for {city}")

    airport_df = pd.json_normalize(airport_data, record_path=['Airport_Info'], meta='City')
    return airport_df


In [12]:
airport=get_airport_data(cities)

NameError: name 'cities' is not defined

In [ ]:
# Example usage:
cities = ['Munich', 'Berlin', 'Cologne', 'Hamburg', 'Frankfurt', 'Leipzig', 'Hannover']
airport_df = get_airport_data(cities)
airport_df


In [14]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def extract_coordinates(html_doc):
    soup = BeautifulSoup(html_doc, 'html.parser')
    # Find infobox
    infobox = soup.find('table', class_='infobox')
    if infobox:
        # Find rows in infobox
        rows = infobox.find_all('tr')
        for row in rows:
            # Check if row contains 'Coordinates' or similar text
            if 'coordinates' in row.text.lower():
                # Extract coordinates from the row
                coordinates = row.find('span', class_='geo').text
                # Split coordinates into latitude and longitude
                lat, lon = map(float, coordinates.split(';'))
                return lat, lon
    # If coordinates not found
    return float('nan'), float('nan')


def get_airport_info(latitude, longitude):
    url = "https://aerodatabox.p.rapidapi.com/airports/search/location"

    querystring = {
        "lat": str(latitude),
        "lon": str(longitude),
        "radiusKm": "50",
        "limit": "10",
        "withFlightInfoOnly": "True"
    }

    headers = {
        "X-RapidAPI-Key": "14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4",
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        return response.json()
    else:
        return None

def get_airport_data(cities):
    airport_data = []

    for city in cities:
        # Construct the Wikipedia URL for the city
        URL = f"https://en.wikipedia.org/wiki/{city}"
        
        # Retrieve the web page content
        web_page = requests.get(URL).text
        
        # Extract coordinates using the function
        lat, lon = extract_coordinates(web_page)
        
        # Get airport information using the coordinates
        airport_info = get_airport_info(lat, lon)
        
        # Append city and airport information to the list
        if airport_info and isinstance(airport_info['items'], list):
            airport_data.append({
                'City': city,
                'Airport_Info': airport_info
            })

    return  pd.json_normalize(airport_data, record_path=['Airport_Info', 'items'], meta='City')
     


In [15]:
cities = ['Munich', 'Berlin', 'Cologne', 'Hamburg', 'Frankfurt', 'Leipzig', 'Hannover']
airport = get_airport_data(cities)

In [9]:
airport= airport.rename(columns = {"location.lat" : "lat"}) 
airport= airport.rename(columns = {"location.lon" : "lon"}) 

In [17]:
airport

,icao,iata,name,shortName,municipalityName,countryCode,location.lat,location.lon,City
0,EDDM,MUC,Munich,Munich,Munich,DE,48.35380,11.786100,Munich
1,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,52.35139,13.493889,Berlin
2,EDDK,CGN,Cologne Bonn,Bonn,Cologne,DE,50.86590,7.142739,Cologne
3,EDDL,DUS,Duesseldorf Düsseldorf,Düsseldorf,Duesseldorf,DE,51.28950,6.766779,Cologne
4,EDDH,HAM,Hamburg,Hamburg,Hamburg,DE,53.63040,9.988229,Hamburg
5,EDDF,FRA,Frankfurt-am-Main,Frankfurt-am-Main,Frankfurt-am-Main,DE,50.02640,8.543129,Frankfurt
6,EDDP,LEJ,Leipzig Halle,Halle,Leipzig,DE,51.43240,12.241600,Leipzig
7,EDDV,HAJ,Hanover Hannover,Hannover,Hanover,DE,52.46110,9.685079,Hannover


# flight_arrival

## get_icao_list

In [4]:
# get the list of icao from the airport table 
def get_icao_list(airport):
    return airport["icao"]

In [12]:
imen = get_icao_list(airport) 
imen

0    EDDM
1    EDDB
2    EDDK
3    EDDL
4    EDDH
5    EDDF
6    EDDP
7    EDDV
Name: icao, dtype: object

## flight_arrival function

### create the flight arrivals using json.normalize # from solution 

In [6]:
def tomorrows_flight_arrivals(icao_list):

    api_key = "14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4"

    berlin_timezone = timezone('Europe/Berlin')
    today = datetime.now(berlin_timezone).date()
    tomorrow = (today + timedelta(days=1))

    list_for_arrivals_df = []

    for icao in icao_list:

        times = [["00:00","11:59"],["12:00","23:59"]]

        for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"

            querystring = {"direction":"Arrival","withCancelled":"false"}

            headers = {
                "X-RapidAPI-Key": "12c7bd34d2msha978e1852a9e04dp17b222jsnd5260f196022",
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
                }

            response = requests.request("GET", url, headers=headers, params=querystring)
            flights_resp = response.json()

            arrivals_df = pd.json_normalize(flights_resp["arrivals"])[["number", "airline.name", "movement.scheduledTime.local", "movement.terminal", "movement.airport.name", "movement.airport.icao"]]
            arrivals_df = arrivals_df.rename(columns={"number": "flight_number", "airline.name": "airline", "movement.scheduledTime.local": "arrival_time", "movement.terminal": "arrival_terminal", "movement.airport.name": "departure_city", "movement.airport.icao": "departure_airport_icao"})
            arrivals_df["arrival_airport_icao"] = icao
            arrivals_df["data_retrieved_on"] = datetime.now().date()
            arrivals_df = arrivals_df[["arrival_airport_icao", "flight_number", "airline", "arrival_time", "arrival_terminal", "departure_city", "departure_airport_icao", "data_retrieved_on"]]

            # fixing arrival_time
            arrivals_df["arrival_time"] = arrivals_df["arrival_time"].str.split("+").str[0]

            list_for_arrivals_df.append(arrivals_df)

    return pd.concat(list_for_arrivals_df, ignore_index=True)

icao_list1 = ["EDDF", "EDDB"]
tomorrows_flight_arrivals(icao_list1)


KeyError: 'arrivals'

### create the flight arrivals using get. as for my icao list there is no terminal info 

In [ ]:
x["arrivals"][0]["aircraft"]["model"]

In [13]:
def tomorrows_flight_arrivals(icao_list):
    api_key = "14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4"
    berlin_timezone = timezone('Europe/Berlin')
    today = datetime.now(berlin_timezone).date()
    tomorrow = (today + timedelta(days=1))

    list_for_arrivals_df = []

    for icao in icao_list:
        times = [["00:00","11:59"],["12:00","23:59"]]

        for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
            querystring = {"direction":"Arrival","withCancelled":"false"}

            headers = {
                "X-RapidAPI-Key": "12c7bd34d2msha978e1852a9e04dp17b222jsnd5260f196022",
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }

            response = requests.request("GET", url, headers=headers, params=querystring)
            flights_resp = response.json()

            if "arrivals" in flights_resp:
                arrivals = flights_resp["arrivals"]
                for arrival in arrivals:
                    flight_number = arrival.get("number")
                    airline = arrival.get("airline", {}).get("name")
                    scheduled_time = arrival.get("movement", {}).get("scheduledTime", {}).get("local")
                    terminal = arrival.get("movement", {}).get("terminal")
                    departure_city = arrival.get("movement", {}).get("airport", {}).get("name")
                    departure_icao = arrival.get("movement", {}).get("airport", {}).get("icao")
                    aircraft = arrival.get("aircraft", {}).get("model")

                    arrival_data = {
                        "arrival_airport_icao": icao,
                        "flight_number": flight_number,
                        "airline": airline,
                        "arrival_time": scheduled_time,
                        "arrival_terminal": terminal,
                        "departure_city": departure_city,
                        "departure_airport_icao": departure_icao,
                        "data_retrieved_on": datetime.now().date(),
                        "aircraft" : aircraft
                    }

                    list_for_arrivals_df.append(arrival_data)
            else:
                print(f"No arrivals found for {icao} at {tomorrow}T{time[0]} - {tomorrow}T{time[1]}")

    return pd.DataFrame(list_for_arrivals_df)


In [15]:

icao_list1 = get_icao_list(airport)
flight_arrivals = tomorrows_flight_arrivals(icao_list1)

In [17]:
flight_arrivals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3118 entries, 0 to 3117
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   arrival_airport_icao    3118 non-null   object
 1   flight_number           3118 non-null   object
 2   airline                 3118 non-null   object
 3   arrival_time            3118 non-null   object
 4   arrival_terminal        3026 non-null   object
 5   departure_city          3118 non-null   object
 6   departure_airport_icao  3106 non-null   object
 7   data_retrieved_on       3118 non-null   object
 8   aircraft                3118 non-null   object
dtypes: object(9)
memory usage: 219.4+ KB


In [34]:
flight_arrivals.sample()

,arrival_airport_icao,flight_number,airline,arrival_time,arrival_terminal,departure_city,departure_airport_icao,data_retrieved_on,aircraft
2936,EDDF,KM 2804,Air Malta,2024-03-15 19:55+01:00,1,Luqa,LMML,2024-03-14,Airbus A320


In [35]:
flight_arrivals['arrival_time'] = pd.to_datetime(flight_arrivals['arrival_time'])
flight_arrivals['data_retrieved_on'] = pd.to_datetime(flight_arrivals['data_retrieved_on'])

flight_arrivals= flight_arrivals.rename(columns = {"arrival_airport_icao" : "icao"}) 

push to sql

In [39]:
schema = "sql_workshop"
host = "127.0.0.1"
user = "root"
password = "188782Im"
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [44]:
flight_arrivals.to_sql('flight_arrivals',
                  if_exists='append',
                  con=connection_string,
                  index=False)

3215

In [43]:
airport.to_sql('airport',
                  if_exists='append',
                  con=connection_string,
                  index=False)

8